In [1]:
print("Um cuspidor esta nascendo!")

Um cuspidor esta nascendo!


In [2]:
!pip install gradio duckdb pandas --quiet


In [214]:
import duckdb
import pandas as pd
url = 'https://raw.githubusercontent.com/alura-cursos/llamaIndex_pandas_query/refs/heads/main/Dados/vendas.csv'
con = duckdb.connect(database="/tmp/data_spitter", read_only=False)

con.execute(f"USE data_spitter")
con.execute(f"CREATE OR REPLACE TABLE vendas_dk AS SELECT * FROM read_csv_auto('{url}')")
#con.sql(f'SELECT ID_compra FROM "{url}" LIMIT 5')
df = pd.read_csv(url)
#con.register("temp_df", df)
#con.execute("CREATE OR REPLACE TABLE vendas AS SELECT * FROM temp_df")
#con.execute(f"SELECT * FROM read_csv_auto('{url}', header=True) LIMIT 5").fetchdf()

# Testa a leitura
#print(con.execute("SELECT ID_compra  FROM vendas LIMIT 5").fetchdf())

In [165]:
print(con.execute("SELECT ID_compra  FROM data_spitter.vendas_dk LIMIT 5").fetchdf())

     ID_compra
0  750-67-8428
1  226-31-3081
2  631-41-3108
3  123-19-1176
4  373-73-7910


In [215]:
#con.execute("INSTALL ducklake;")
con.execute("DETACH DATABASE IF EXISTS my_ducklake;")
con.execute("ATTACH 'ducklake:metadata.ducklake' AS my_ducklake (DATA_PATH '/tmp/dklq-data-spitter') ;")
#con.execute("USE my_ducklake;")
#con.execute("CREATE OR REPLACE TABLE vendas_duck AS SELECT * FROM temp_df")
#con.execute("SELECT * FROM vendas_duck").fetchall()

In [216]:
#dklq = SpitterDucklakeOperator()
df = con.execute("SELECT * FROM tmp.").df()

#dklq.create_or_replace_table(df, "vendas_duck")
#df


aqui


,ID_compra,filial,cidade,tipo_cliente,genero,tipo_produto,preco_unitario,quantidade,imposto_5%,total,data,hora,forma_pagamento,avaliacao
0,750-67-8428,A,Santo André,Membro,Feminino,Saúde e Beleza,74.69,7,26.1415,548.9715,2024-01-05,13:08:00,Carteira Digital,9.1
1,226-31-3081,C,São Caetano,Normal,Feminino,Eletrônicos,15.28,5,3.8200,80.2200,2024-03-08,10:29:00,Dinheiro,9.6
2,631-41-3108,A,Santo André,Normal,Masculino,Casa,46.33,7,16.2155,340.5255,2024-03-03,13:23:00,Cartão de Crédito,7.4
3,123-19-1176,A,Santo André,Membro,Masculino,Saúde e Beleza,58.22,8,23.2880,489.0480,2024-01-27,20:33:00,Carteira Digital,8.4
4,373-73-7910,A,Santo André,Normal,Masculino,Esportes e Viagem,86.31,7,30.2085,634.3785,2024-02-08,10:37:00,Carteira Digital,5.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,233-67-5758,C,São Caetano,Normal,Masculino,Saúde e Beleza,40.35,1,2.0175,42.3675,2024-01-29,13:46:00,Carteira Digital,6.2
996,303-96-2227,B,São Bernardo do Campo,Normal,Feminino,Casa,97.38,10,48.6900,1022.4900,2024-03-02,17:16:00,Carteira Digital,4.4
997,727-02-1313,A,Santo André,Membro,Masculino,Alimentos e Bebidas,31.84,1,1.5920,33.4320,2024-02-09,13:22:00,Dinheiro,7.7
998,347-56-2442,A,Santo André,Normal,Masculino,Casa,65.82,1,3.2910,69.1110,2024-02-22,15:33:00,Dinheiro,4.1


In [236]:
import duckdb
class SpitterDucklakeOperator():
    def __init__(self,con,  path=None):

      self.ducklake_db = "spitter_ducklake"
      self.path = path if path else f"/tmp/dklq_{self.ducklake_db}"
      self.con = con
      self.init_ducklake()


    def init_ducklake(self):
      try:
        #con = duckdb.connect(database=':memory:')
        self.con.execute("INSTALL ducklake;")
        self.con.execute(f"DETACH DATABASE IF EXISTS {self.ducklake_db};")
        self.con.execute(f"ATTACH 'ducklake:metadata.ducklake' AS {self.ducklake_db} (DATA_PATH '{self.path}');")
        #return con
      except Exception as e:
        raise e

    def create_or_replace_table(self, source_df, output_table_name, layer, pk_key=False):
      try:
        if pk_key:
          pass
          #con.execute(f"CREATE OR REPLACE TABLE {output_table_name} AS SELECT * FROM {source_df} WHERE {pk_key} IS NOT NULL")
        else:
          self.con.execute(f"CREATE SCHEMA IF NOT EXISTS {self.ducklake_db}.{layer}")
          self.con.execute(f"CREATE OR REPLACE TABLE {self.ducklake_db}.{layer}.{output_table_name} AS SELECT * FROM {source_df}")
        return True, "success"
      except Exception as e:
        return False,str(e)

    def insert_table(self, source_df, output_table_name, layer):
      try:
        self.con.execute(f"CREATE SCHEMA IF NOT EXISTS {self.ducklake_db}.{layer}")
        self.con.execute(f"INSERT INTO {self.ducklake_db}.{output_table_name} AS SELECT * FROM {source_df}")
        return True, "success"
      except Exception as e:
        return False,str(e)

    def upsert_table(self, source_df, output_table_name,layer, pk_key):
      try:
        self.con.execute(f"CREATE SCHEMA IF NOT EXISTS {self.ducklake_db}.{layer}")
        self.con.execute(f"INSERT INTO {self.ducklake_db}.{output_table_name} AS SELECT * FROM {source_df}")
        return True, "success"
      except Exception as e:
        return False,str(e)




In [234]:
#csv operations
import chardet
import duckdb


class SpitterOperator():
    def __init__(self, file):
        self.df = pd.DataFrame()
        self.primary_key = ""
        self.cols = []
        self.con = duckdb.connect(database=':memory:')
        self.dklq = SpitterDucklakeOperator(self.con)
        self.table_name = ""
        self.op_mode = ""
        if file is not None:
            self.load_bronze(file)
        else:
            return "Envie um file CSV."

    def preview_bronze(self):
        preview_df = self.con.execute(f"""
          SELECT * FROM temp.tmp_{self.table_name} LIMIT 10
        """).fetchdf()
        return preview_df

    def set_primary_key(self, column_name):
        if column_name in self.df.columns:
            self.primary_key = column_name

    def set_table_op_mode(self, op_mode):
        self.op_mode = op_mode

    def table_exists(self, table_name):
      return False
        # try:
        #     self.con.execute(f"SELECT * FROM {table_name} LIMIT 0")
        #     return True
        # except Exception as e:
        #     return False

    def persist_table_on_dklq(self, layer):
      if self.table_exists(self.table_name):
          #self.dklq.upsert_table(f"tmp_{self.table_name}", self.table_name, self.primary_key)
          self.dklq.insert_table(f"tmp_{self.table_name}", f"tb_{layer}_{self.table_name}", layer)
          if not result[0]:
            raise Exception(result[1])
      else:
          result = self.dklq.create_or_replace_table(f"tmp_{self.table_name}", f"tb_{layer}_{self.table_name}",layer, self.primary_key)
          if not result[0]:
            raise Exception(result[1])


    def load_bronze(self,file):

        try:
          self.table_name = file.name.split("/")[-1].split(".")[0]
          self.con.execute(f"""
              CREATE TEMPORARY TABLE tmp_{self.table_name} AS
              SELECT * FROM read_csv_auto('{file.name}', header=True)
          """)
          self.cols = [row[0] for row in self.con.execute(f"DESCRIBE tmp_{self.table_name}").fetchall()]


          #op = self.get_file_options(file.name)
          # self.df = pd.read_csv(file.name,
          #                      # delimiter=op["encoding"],
          #                       #encoding=op["encoding"],
          #                       skipinitialspace=True,
          #                       quotechar='"'
          #                       )


          #self.cols = list(self.df.columns)



          #return self.df
        except Exception as e:
          raise e



In [235]:
import gradio as gr

def init_file_operator(file):
    spitter_operator = SpitterOperator(file)
    return spitter_operator, "bronze"

def preview_bronze(spitter_operator):
    return spitter_operator.preview_bronze()

def set_pk_column(spitter_operator, pk_column):
    if spitter_operator is not None:
        spitter_operator.set_primary_key(pk_column)
    return spitter_operator

#button options
def set_pk_chk_visibility(spitter_operator, checkbox_value):
    if checkbox_value:
        return gr.update(interactive=True, visible=True,choices=spitter_operator.cols)
    else:
        return gr.update(interactive=False, visible=False)

def set_op_chk_visibility(spitter_operator):
    return gr.update(interactive=True, visible=True,choices=["upsert","append","overwrite"])


def set_table_op_mode_btn(spitter_operator, op_mode):
  spitter_operator.set_table_op_mode(op_mode)
  return f"Commit - table -> {spitter_operator.table_name} | op_mode -> {op_mode}", True

def publish_bronze_table(spitter_operator, commit_state, layer):
  if commit_state:
    spitter_operator.persist_table_on_dklq(layer)
    return "sucess"
  else:
    return "Error. Please check your settings!"




with gr.Blocks(title="Arquitetura Medalhão - Data Spitter - CSV - multi Reads") as demo:
    gr.Markdown("#Arquitetura Medalhão (Bronze / Silver / Gold)")
    gr.Markdown("# Spitter Data Lakehouse Explorer")
    gr.Markdown("Leia seus dados, configure as operacoes, e maos a obra. Apresentamos suporte a cargas incrementais e de sobrescrita")
    gr.Markdown("As tabelas criadas utilizarao o nome do arquivo como referencia")
    st_spitter_operator = gr.State(None)
    st_layer = gr.State("")

#with gr.Row():
    #BronzeLayer
    with gr.Tab("Bronze Layer"):
        gr.Markdown("### Upload dos dados brutos")
        st_choices = gr.State([])
        st_commit_bronze = gr.State(False)

        csv_bronze = gr.File(label="Selecione um CSV",file_count="single", type="filepath")
        csv_bronze.upload(init_file_operator, inputs=csv_bronze, outputs=[st_spitter_operator,st_layer])

        gr.Markdown("### Preview dos dados brutos")
        bronze_btn = gr.Button("Preview Bronze")
        bronze_table = gr.DataFrame(label="Bronze Data")
        bronze_btn.click(preview_bronze, inputs=st_spitter_operator, outputs=bronze_table)

        pk_checkbox = gr.Checkbox(label="Does the table have a primary key?", value=False)
        pk_dropdown = gr.Dropdown(label="Select the primary key", choices=[], interactive=False, visible=False)
        pk_checkbox.change(set_pk_chk_visibility, inputs=[st_spitter_operator, pk_checkbox], outputs=pk_dropdown)
        pk_dropdown.change(set_pk_column, inputs=[st_spitter_operator, pk_dropdown], outputs=st_spitter_operator)


        op_dropdown = gr.Dropdown(label="Select the table operaration mode", choices=[], interactive=False, visible=False)
        st_spitter_operator.change(set_op_chk_visibility,inputs=st_spitter_operator,outputs=op_dropdown)

        output_text = gr.Markdown("") #gr.Textbox(value="Mensagem de commit", placeholder="Aguardando configuracoes")
        op_dropdown.change(set_table_op_mode_btn, inputs=[st_spitter_operator, op_dropdown], outputs=[output_text, st_commit_bronze])

        alerta_md_component = gr.Markdown("")
        publish_bronze_btn = gr.Button(value=f"Commit", visible=True)
        publish_bronze_btn.click(publish_bronze_table, inputs=[st_spitter_operator, st_commit_bronze, st_layer], outputs=alerta_md_component)


        #
        #st_spitter_operator.change(set_publish_bronze_btn, inputs=[st_spitter_operator], outputs=publish_bronze_btn)








    # # Camada Silver
    # with gr.Tab("Camada Silver"):
    #     gr.Markdown("### Limpeza e padronização dos dados")
    #     botao_silver = gr.Button("Transformar Bronze → Silver")
    #     silver_tabela = gr.DataFrame(label="Prévia dos dados Silver")
    #     botao_silver.click(transformar_para_silver, outputs=silver_tabela)

    # # Camada Gold
    # with gr.Tab("Camada Gold"):
    #     gr.Markdown("### Agregações e métricas finais")
    #     botao_gold = gr.Button("Transformar Silver → Gold")
    #     gold_tabela = gr.DataFrame(label="Tabela final Gold")
    #     botao_gold.click(transformar_para_gold, outputs=gold_tabela)

demo.launch(debug=True)


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://035ff33f1bc78539b0.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/uvicorn/protocols/http/h11_impl.py", line 403, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/uvicorn/middleware/proxy_headers.py", line 60, in __call__
    return await self.app(scope, receive, send)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/fastapi/applications.py", line 1133, in __call__
    await super().__call__(scope, receive, send)
  File "/usr/local/lib/python3.12/dist-packages/starlette/applications.py", line 113, in __call__
    await self.middleware_stack(scope, receive, send)
  File "/usr/local/lib/python3.12/dist-packages/starlette/middleware/errors.py", line 186, in __call__
    raise exc
  File "/usr/local/lib/python3.12/dist-packages/starlette/middleware/errors.py",

Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7865 <> https://035ff33f1bc78539b0.gradio.live


In [155]:
import gradio as gr

def selecionar_pk(valor):
    # valor é o item selecionado no dropdown
    return f"Coluna escolhida: {valor}"

with gr.Blocks() as demo:
    dropdown_pk = gr.Dropdown(
        label="Selecione a PK",
        choices=["id", "nome", "email"],
        interactive=True
    )
    output_text = gr.Textbox()

    # Captura seleção
    dropdown_pk.change(selecionar_pk, inputs=[dropdown_pk], outputs=[output_text])

demo.launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://78570c64b388a3bcff.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
